In [ ]:
import json

h2_open = open("../log-simple-cloudflare" + "/h2-performances.json", "r")
h3_open = open("../log-simple-cloudflare" + "/h3-performances.json", "r")

h2_load = json.load(h2_open)
h3_load = json.load(h3_open)

h2_open.close()
h3_open.close()



In [ ]:
from itertools import chain


h2_entrieses = [{"number": x["number"], **x["entries"][0]} for x in h2_load]
h2_entries = entries = pd.DataFrame(chain(h2_entrieses))
h3_entrieses = [{"number": x["number"], **x["entries"][0]} for x in h3_load]
h3_entries = entries = pd.DataFrame(chain(h3_entrieses))

difference = h3_entries.select_dtypes(include=float) / h2_entries.select_dtypes(
    include=float
)
difference = difference.replace([np.inf, -np.inf], np.nan)
difference = difference.dropna()
difference.mean()